In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys
import os
# Añadir el directorio raíz del proyecto al path para importar módulos
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Importaciones de los módulos del proyecto
from src.db_manager import SQLiteWrapper
from src.functions_dropbox import manage_epub_metadata
from src.data_processor import process_data
from src.functions_notion import create_books, create_annotations, create_book_pages
from src.config import NOTION_API_TOKEN, NOTION_BOOKS_DATABASE_ID, NOTION_ANNOTATIONS_DATABASE_ID
from notion_client import Client

pd.options.display.max_columns = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Carga de BBDD

In [6]:
db_path = os.path.join("..", "data", "KoboReader.sqlite") 
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión
anotaciones_df = db.get_annotations()
libros_ereader_df = db.get_books()
# db.close()

# Dropbox

In [ ]:
# Obtener metadatos de Dropbox con caché inteligente
epub_metadata = manage_epub_metadata(
    libros_ereader_df, 
    cache_path=os.path.join("..", "data", "epub_metadata.pkl"),
    folder_path='/Aplicaciones/Rakuten Kobo'
)

Fallo en The_Rational_Male.epub
'title'
Fallo en Left_Swipes_Love_A_Millennials_Guide_to_Hookups_Dating_and_.epub
'title'
Error al procesar el archivo toc.ncx: No se encontró ningún archivo toc.ncx en el EPUB.
Fallo en How to Win at Chess_ The Ultimate Guide for Beginners and -- Levy Rozman -- 2023 -- Clarkson Potter_Ten Speed -- 1984862081 -- cb65fc4e7be857658c8b8c3e11a93cae -- Anna’s Archive.epub
'title'
Fallo en SUMMARYThe_Six_Pillars_of_SelfEsteem_by_Nathaniel_Branden.epub
'title'
Fallo en Hill - Stone - La Actitud Mental Positiva -- Edited by Fabricio Bossio, ELEVEN - Biblioteca del Nuevo -- b5290d34663758be225b9aaf114623a2 -- Anna’s Archive.epub
'title'
Fallo en The_Slight_Edge_Turning_Simple_Disciplines_into_Massive_Success_.epub
'title'
Fallo en Grit_Angela_Duckworth.epub
'title'
Fallo en Models -- Mark Manson -- Mark Manson -- 62f7c7c925b50cdbfdbafc78bb65ec1e -- Anna’s Archive.epub
'title'
Fallo en How_to_Stop_Worrying_and_Start_Living.epub
'title'
Fallo en Google Cloud Certif

In [ ]:
# Procesar y enriquecer los datos usando la función modularizada
libros_df = process_data(anotaciones_df, libros_ereader_df, epub_metadata)
libros_df.head()

# Exportacion a Notion

In [ ]:
# Configuración inicial del cliente de Notion
notion = Client(auth=NOTION_API_TOKEN)

# 1. Crear/actualizar los libros en la base de datos de libros
print("📚 Sincronizando libros...")
create_books(libros_df, notion, NOTION_BOOKS_DATABASE_ID)

Fallo en ('La playa de los ahogados', 'Domingo Villar')
Request to Notion API has timed out
Traceback (most recent call last):
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpx\_transports\default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpcore\_sync\connection_pool.py", line 256, in handle_request
    raise exc from None
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpcore\_sync\connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpcore\_sync\connection.

In [ ]:
# 2. Crear solo las anotaciones nuevas
print("\n📝 Sincronizando anotaciones...")
create_annotations(anotaciones_df, notion, NOTION_ANNOTATIONS_DATABASE_ID, NOTION_BOOKS_DATABASE_ID)

In [ ]:
# 3. Actualizar el contenido de las páginas de los libros (solo si hay cambios)
print("\n📖 Actualizando páginas de libros...")
create_book_pages(anotaciones_df, notion, NOTION_BOOKS_DATABASE_ID)

Contenido existente eliminado para el libro '12 reglas para vivir'.
Contenido existente eliminado para el libro 'A puerta cerrada'.
Contenido existente eliminado para el libro 'Acontece que no es poco'.
Contenido existente eliminado para el libro 'Amanecer'.
Contenido existente eliminado para el libro 'Attached: Are you Anxious, Avoidant or Secure? How the science of adult attachment can help you find – and keep – love'.
Contenido existente eliminado para el libro 'Breaking the Habit of Being Yourself'.
Contenido existente eliminado para el libro 'Como hacer que te pasen cosas buenas'.
Contenido existente eliminado para el libro 'Cómo invertir en Bolsa a largo plazo partiendo de cero'.
Contenido existente eliminado para el libro 'Cómo invertir en fondos de inversión con sentido común'.
Contenido existente eliminado para el libro 'Cómo piensan los ricos'.
Contenido existente eliminado para el libro 'Educación financiera avanzada partiendo de cero'.
Contenido existente eliminado para el 